# ASSIGNMENT:

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

models_path="dataset/models/"
scenes_path="dataset/scenes/"

products_a=[f"{models_path}ref{i}.png" for i in range(1,15)]
products_b=[f"{models_path}ref{i}.png" for i in range(15,28)]

scenes_a=[f"{scenes_path}scene{i}.png" for i in range(1,6)]
scenes_b=[f"{scenes_path}scene{i}.png" for i in range(6,13)]

In [7]:
product_descriptors_a = []
for product in products_a:
    image = cv2.imread(product, 0)
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints = sift.detect(image)
    product_descriptors_a.append(list(sift.compute(image, keypoints))+[image.shape])

In [8]:
settings={}

def denoise_image(image,  filter_param,type,prefilter=False, prefilter_param=None):
    if prefilter:
        k_size = 9
        mean_kernel = np.ones([k_size, k_size])/(k_size**2)
        # image = cv2.filter2D(image, -1, mean_kernel)
        image = cv2.filter2D(**(prefilter_param))
        settings['prefilter']={'k_size':k_size}
    match type:
        case "bilateral":
            k = 30
            sigma1 = 1500
            sigma2 = 250  # non so se vanno bene
            settings['bilateral']= filter_param # {'k':k,'sigma1':sigma1,'sigma2':sigma2}
            # cv2.bilateralFilter(image, k, sigma1, sigma2)
            return cv2.bilateralFilter(image, **(filter_param)), filter_param
        case 'gaussian':
            sigma = 3
            k_size = 2*int(np.ceil(3*sigma)) + 1
            settings['gaussian']={'sigma':sigma,"k_size":k_size}
            return cv2.GaussianBlur(image, **(filter_param)), filter_param
        case _:
            return image

In [9]:
def print_single_instances(results: dict):
    for scene in results.keys():
        print(f'Scene {scene}:')
        for product in results[scene].keys():
            print(f'\tProduct {product}'
                  +f', {results[scene][product]["count"]} matches:' # <-- comment line if counts are not needed
                  +f' {results[scene][product]["geometry"]}'
                  +''
                )
            
def diagonal(points):
    return np.sqrt(np.sum(np.subtract(points[:1,:],points[1:,:])**2))

def get_geometry(corners):
    import math
    return {'position': tuple(np.round((np.sum(corners,axis=0)/4)[0],0).astype(int)),
        'width': f'{diagonal(corners[:2,:]):0.0f}px',
        'height': f'{diagonal(corners[1:3,:]):0.0f}px'
    }

In [12]:
from sklearn.model_selection import ParameterGrid

SHOW_FILTERED=False
scene_descriptors_a = []

pg = ParameterGrid({
    'd' : [*range(5,10)],
    'sigmaColor' : [*range(15,900, 75)],
    'sigmaSpace' : [*range(15,900, 75)]
})

prefilter_param = {}

for i, scene in enumerate(scenes_a):
    results = {}
    for j in pg:
        image = cv2.imread(scene,cv2.IMREAD_GRAYSCALE)
        filtered_image, settings = denoise_image(image,type='bilateral',filter_param=j)
        sift = cv2.xfeatures2d.SIFT_create()
        keypoints = sift.detect(filtered_image)
        scene_descriptors = sift.compute(filtered_image, keypoints)
        if SHOW_FILTERED:
            img_kp=cv2.drawKeypoints(filtered_image,keypoints,None,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            plt.figure(figsize=(20,10))
            plt.axis('off')
            # plt.imshow(filtered_image,cmap='gray')
            plt.imshow(img_kp)
            plt.title(scene[len(models_path):])
            plt.show()

        MIN_MATCH_COUNT = 50
        results[i] = {
            # 'products': []
        }   
        for j, product in enumerate(product_descriptors_a, 1):
            FLANN_INDEX_KDTREE = 1
            index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
            search_params = dict(checks=50)

            flann = cv2.FlannBasedMatcher(index_params, search_params)
            matches = flann.knnMatch(product[1], scene_descriptors[1], k=2)

            good = []
            for m, n in matches:
                if m.distance < 0.7*n.distance:
                    good.append(m)
            num_matches = len(good)

            try: # da errori del cazzo non lo so :)
                src_pts = np.float32(
                    [product[0][m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
                dst_pts = np.float32(
                    [scene_descriptors[0][m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
                
                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

                matchesMask = mask.ravel().tolist()

                h, w = product[2]
                pts = np.float32([[0, 0], [0, h-1], [w-1, h-1],
                                [w-1, 0]]).reshape(-1, 1, 2)

                dst = np.int32(cv2.perspectiveTransform(pts, M))
                # print(len(dst[0]))

                # settings['min_match']=MIN_MATCH_COUNT
                if num_matches >= MIN_MATCH_COUNT:
                    results[i][j] = {
                        'count': num_matches,
                        'geometry': get_geometry(dst)
                    }
            except: cv2.error
        display(settings)
        print_single_instances(results)
        settings = {}

{'d': 5, 'sigmaColor': 15, 'sigmaSpace': 15}

Scene 0:
	Product 1, 471 matches: {'position': (412, 538), 'width': '1050px', 'height': '801px'}
	Product 2, 323 matches: {'position': (1247, 533), 'width': '1049px', 'height': '802px'}
	Product 5, 80 matches: {'position': (402, 626), 'width': '1092px', 'height': '822px'}
	Product 14, 66 matches: {'position': (1230, 544), 'width': '964px', 'height': '743px'}


{'d': 5, 'sigmaColor': 15, 'sigmaSpace': 90}

Scene 0:
	Product 1, 469 matches: {'position': (411, 539), 'width': '1049px', 'height': '801px'}
	Product 2, 338 matches: {'position': (1248, 534), 'width': '1046px', 'height': '805px'}
	Product 5, 72 matches: {'position': (366, 410), 'width': '828px', 'height': '448px'}
	Product 14, 65 matches: {'position': (1230, 537), 'width': '948px', 'height': '736px'}


{'d': 5, 'sigmaColor': 15, 'sigmaSpace': 165}

...

{'d': 5, 'sigmaColor': 315, 'sigmaSpace': 765}

Scene 0:
	Product 1, 524 matches: {'position': (412, 538), 'width': '1049px', 'height': '801px'}
	Product 2, 345 matches: {'position': (1248, 533), 'width': '1048px', 'height': '803px'}
	Product 5, 66 matches: {'position': (366, 408), 'width': '826px', 'height': '447px'}
	Product 14, 70 matches: {'position': (1236, 512), 'width': '1034px', 'height': '645px'}


{'d': 5, 'sigmaColor': 315, 'sigmaSpace': 840}

Scene 0:
	Product 1, 522 matches: {'position': (411, 538), 'width': '1050px', 'height': '801px'}
	Product 2, 335 matches: {'position': (1247, 533), 'width': '1049px', 'height': '804px'}
	Product 5, 69 matches: {'position': (384, 406), 'width': '814px', 'height': '444px'}
	Product 14, 68 matches: {'position': (1230, 522), 'width': '915px', 'height': '717px'}


KeyboardInterrupt: 

scene1:1,2  
scene2:3,4,5  
scene3:6,7,8  
scene4:1,2,8,9,10  
scene5:2,11,12  

## TRACK B